In [1]:
#import# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf

from tensorflow import keras

inputs = keras.layers.Input(shape=(224,224,3), name="image_input")
x = keras.layers.Conv2D(64, (3,3), padding='same')(inputs)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.ReLU()(x)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(10, activation='softmax')(x)

model = keras.models.Model(inputs=inputs, outputs=outputs)

print(model.summary())

2025-06-23 11:37:37.248956: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-23 11:37:37.258061: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-23 11:37:37.268741: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8473] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-23 11:37:37.272292: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1471] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-23 11:37:37.280239: I tensorflow/core/platform/cpu_feature_guar

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image_input (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 batch_normalization (Batch  (None, 224, 224, 64)      256       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 224, 224, 64)      0         
                                                                 
 global_average_pooling2d (  (None, 64)                0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 10)                650   

I0000 00:00:1750678658.694713    8469 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1750678658.737520    8469 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1750678658.745074    8469 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1750678658.748456    8469 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
import warnings

# 'my_problematic_module'에서 발생하는 모든 경고를 무시합니다.
warnings.filterwarnings('ignore', module='my_problematic_module')

In [4]:
import tensorflow as tf
print("GPUs:", tf.config.list_physical_devices('GPU'))
print("cuDNN enabled:", tf.test.is_built_with_cuda(), tf.test.is_built_with_gpu_support())


GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
cuDNN enabled: True True


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import time

# ✅ TensorFlow 및 GPU 확인
print("TensorFlow version:", tf.__version__)
print("GPU devices:", tf.config.list_physical_devices('GPU'))

# ✅ 고해상도 가짜 이미지 데이터 생성 (10000개, 256x256 RGB)
x_train = np.random.rand(100, 256, 256, 3).astype(np.float32)
y_train = np.random.randint(0, 100, 100).astype(np.int32)  # 클래스 100개

# ✅ 고부하 CNN 모델 정의 (ResNet 스타일)
def make_heavy_model():
    model = models.Sequential()
    model.add(layers.Input(shape=(256, 256, 3)))

    for _ in range(6):  # 6개의 블록 → 과도한 풀링 방지
        model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    # Global Average Pooling으로 크기 문제 방지
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(100, activation='softmax'))  # 100-class 분류

    return model

# ✅ 모델 생성 및 컴파일
model = make_heavy_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# ✅ 모델 구조 출력
model.summary()

# ✅ 학습 (GPU 부하 유발: 큰 배치와 반복)
start_time = time.time()
model.fit(x_train, y_train, epochs=5, batch_size=256, validation_split=0.1)
end_time = time.time()

print(f"✅ Training time: {end_time - start_time:.2f} seconds")


TensorFlow version: 2.17.0
GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 256, 256, 64)      1792      
                                                                 
 batch_normalization_1 (Bat  (None, 256, 256, 64)      256       
 chNormalization)                                                
                                                                 
 conv2d_2 (Conv2D)           (None, 256, 256, 64)      36928     
                                                                 
 batch_normalization_2 (Bat  (None, 256, 256, 64)      256       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 128, 128, 64)      0         
 D)            

2025-06-23 11:37:54.692380: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90701
W0000 00:00:1750678674.781180    8548 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678674.797839    8548 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678674.801576    8548 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678674.809620    8548 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678674.815083    8548 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678674.820165    8548 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678674.835207    8548 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678674.840751    8548 gpu_

1/1 [==============================] - ETA: 0s - loss: 4.7431 - accuracy: 0.0111

'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
W0000 00:00:1750678681.077953    8546 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678681.078424    8546 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678681.078862    8546 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678681.079467    8546 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678681.080076    8546 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678681.080676    8546 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678681.081874    8546 gpu_timer.cc:114] Skipping

1/1 [==============================] - 9s 9s/step - loss: 4.7431 - accuracy: 0.0111 - val_loss: 4.6044 - val_accuracy: 0.0000e+00
Epoch 2/5


...

1/1 [==============================] - 1s 1s/step - loss: 4.3756 - accuracy: 0.0444 - val_loss: 4.6035 - val_accuracy: 0.0000e+00
Epoch 4/5


W0000 00:00:1750678686.055013    8550 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678686.057602    8550 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678686.061000    8550 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678686.063717    8550 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678686.066977    8550 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678686.071057    8550 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678686.074484    8550 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678686.077597    8550 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678686.081485    8550 gp

1/1 [==============================] - 1s 1s/step - loss: 4.1931 - accuracy: 0.0444 - val_loss: 4.6032 - val_accuracy: 0.0000e+00
Epoch 5/5


W0000 00:00:1750678687.269921    8549 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678687.270238    8549 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678687.270416    8549 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678687.270579    8549 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678687.270803    8549 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678687.270942    8549 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678687.271098    8549 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678687.271242    8549 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750678687.271374    8549 gp

1/1 [==============================] - 1s 522ms/step - loss: 4.0965 - accuracy: 0.0556 - val_loss: 4.6031 - val_accuracy: 0.0000e+00
✅ Training time: 15.64 seconds


In [6]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.17.0


In [7]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [8]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
predictions = model(x_train[:1]).numpy()
predictions

array([[0.1328891 , 0.16291453, 0.08293683, 0.1262132 , 0.0230473 ,
        0.07909623, 0.15684828, 0.1247583 , 0.05543646, 0.05585982]],
      dtype=float32)

In [10]:
tf.nn.softmax(predictions).numpy()

array([[0.10323976, 0.10638659, 0.09820938, 0.10255285, 0.09250034,
        0.09783293, 0.10574317, 0.10240375, 0.09554539, 0.09558585]],
      dtype=float32)

In [11]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [12]:
loss_fn(y_train[:1], predictions).numpy()

2.324494

In [13]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [14]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5


/usr/local/lib/python3.12/dist-packages/tf_keras/src/backend.py:5729: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


   1/1875 [..............................] - ETA: 13:57 - loss: 2.6336 - accuracy: 0.0312

'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)


1875/1875 [==============================] - 2s 958us/step - loss: 0.3037 - accuracy: 0.9115
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1474 - accuracy: 0.9559
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1101 - accuracy: 0.9665
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0906 - accuracy: 0.9722
Epoch 5/5
1875/1875 [==============================] - 2s 844us/step - loss: 0.0763 - accuracy: 0.9765


In [15]:
model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0671 - accuracy: 0.9783
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0589 - accuracy: 0.9814
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0518 - accuracy: 0.9832
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0497 - accuracy: 0.9834
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0436 - accuracy: 0.9858
313/313 - 0s - loss: 0.0705 - accuracy: 0.9791 - 248ms/epoch - 791us/step


[0.0705113559961319, 0.9790999889373779]

In [16]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [17]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0.08533677, 0.08533677, 0.08533677, 0.08533686, 0.08533677,
        0.08533677, 0.08533677, 0.23196906, 0.08533677, 0.08533679],
       [0.08533675, 0.08533675, 0.23196931, 0.08533675, 0.08533675,
        0.08533675, 0.08533675, 0.08533675, 0.08533675, 0.08533675],
       [0.08535121, 0.23174031, 0.08535153, 0.08535124, 0.08535322,
        0.08535124, 0.08535162, 0.08536451, 0.08543391, 0.08535121],
       [0.23196895, 0.08533677, 0.08533688, 0.08533677, 0.08533677,
        0.08533677, 0.0853368 , 0.08533677, 0.08533677, 0.08533677],
       [0.08534534, 0.08534443, 0.08534443, 0.08534443, 0.23184767,
        0.08534443, 0.08534443, 0.0853449 , 0.08534443, 0.08539551]],
      dtype=float32)>